# Урок 6: Structured Pruning

Физическое удаление целых блоков: голов внимания или полных слоев. Преимущество — ускорение на любом железе без спец-библиотек.

## 1. Математическая теория

### 1.1. Структурная оценка важности
Мы оцениваем важность всей группы параметров (например, группы сверток или головы внимания), используя сумму градиентов или флуктуации активаций.

### 1.2. Методы из обзора:
*   **LLM-Pruner (Ma et al., 2023):** Использует структурную зависимость. Удаляет целые головы, основываясь на первом приближении ошибки (Taylor Expansion) и критерии «целостности» графа.
*   **SliceGPT (Ashkboos et al., 2024):** Экзотический метод. Использует вычислительный трюк (ортогональные матрицы), чтобы сжать информацию в меньшее измерение, после чего «отрезает» лишнее.
*   **Sheared LLaMA (Xia et al., 2024):** Подход, при котором модель сама учится «выбрасывать» ненужные слои во время дообучения на маленьком датасете.
*   **Shortened LLaMA (Kim et al., 2024):** Исследует удаление целых слоев в глубину, показывая, что LLM крайне избыточны по вертикали.

---

In [ ]:
import torch
import torch.nn as nn
from src.model import GPTLanguageModel

def estimate_head_importance_raw(attn_weights):
    # Вычисляем энергию каждой головы
    # attn_weights: (B, H, T, T)
    return attn_weights.abs().mean(dim=(0, 2, 3))

print("nanoGPT Track: Реализована логика оценки важности голов через среднее внимание.")

## 2. Промышленная реализация: Optimum (ONNX Optimization)
Библиотека `optimum` позволяет удалять избыточные узлы и квантовать модель для инференса на CPU/Edge устройствах.

In [ ]:
from optimum.onnxruntime import ORTModelForCausalLM
try:
    model_onnx = ORTModelForCausalLM.from_pretrained("tiny-llama", export=True)
    print("Llama Track: Модель экспортирована в ONNX с автоматической оптимизацией графа.")
except Exception as e:
    print(f"Ошибка: {e}")